In [1]:
import pandas as pd

check_list_df = pd.read_csv('/Volumes/x9/INITIAL_DATABASE/check_list_to_process.csv')
# Display the first few rows to verify import
check_list_df.head()

,Unnamed: 0,sub,strategy,test/train,sex,age,dominancM,dominancO,workingcapacity,besoincogni,...,is static?,Comment,reactive,proactive,static,rolling,mean WC,mean BS,femme,homme
0,1,sub04,reactive,test,f,21,D,D,44,27,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,sub04,static,test,f,21,D,D,44,27,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,sub05,reactive,test,f,24,D,D,30,41,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,sub06,proactive,test,m,22,D,D,35,32,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,sub06,rolling,test,m,22,D,D,35,32,...,0,Left channels 50Hz intermittent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
columns_to_import = [   
    'task','n_block','n_back','trial','letter','responseTime', 'feedback','score'
]

data_df = pd.read_csv('/Volumes/x9/INITIAL_DATABASE/sub04/sub04-reactive-test_ADOEFFORT_2023-12-13_11h01.47.639.csv')

data_df = data_df[columns_to_import]
data_df['n_block'] = data_df['n_block'].bfill()
data_df['n_back'] = data_df['n_back'].bfill()
data_df['rt_h'] = data_df['responseTime'].replace(1.5, pd.NA)
data_df.insert(0, 'sub', 'sub04')
data_df.insert(1, 'strategy', 'reactive')

# adding clicked trials
data_df['click'] = data_df['responseTime'].notna().astype(int)
data_df



,sub,strategy,task,n_block,n_back,trial,letter,responseTime,feedback,score,click
0,sub04,reactive,ax-cpt,0.0,1.0,0.0,m,<NA>,9.0,0,0
1,sub04,reactive,ax-cpt,0.0,1.0,1.0,u,<NA>,9.0,0,0
2,sub04,reactive,ax-cpt,0.0,1.0,2.0,n,<NA>,9.0,0,0
3,sub04,reactive,ax-cpt,0.0,1.0,3.0,c,0.410151,13.0,1,1
4,sub04,reactive,ax-cpt,0.0,1.0,4.0,m,<NA>,9.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1337,sub04,reactive,ax-cpt,21.0,3.0,56.0,m,<NA>,9.0,19,0
1338,sub04,reactive,ax-cpt,21.0,3.0,57.0,u,<NA>,9.0,19,0
1339,sub04,reactive,ax-cpt,21.0,3.0,58.0,n,<NA>,9.0,19,0
1340,sub04,reactive,ax-cpt,21.0,3.0,59.0,c,0.329452,13.0,20,1


In [7]:
import os

all_data = []

columns_to_import = [   
    'task','n_block','n_back','trial','letter','responseTime', 'feedback','score'
]

for idx, row in check_list_df.iterrows():
    sub = row['sub']
    strategy = row['strategy']
    found = False
    for f in os.listdir(f'/Volumes/x9/INITIAL_DATABASE/{sub}/'):
        if f.endswith('.csv') and sub in f and strategy in f and "ADOEFFORT" in f:
            data_df = pd.read_csv(
                os.path.join(f'/Volumes/x9/INITIAL_DATABASE/{sub}/', f),
                usecols=lambda col: col in columns_to_import
            )
            data_df = data_df[columns_to_import]
            data_df['sub'] = sub
            data_df['task'] = 'n-back' if strategy in ['static', 'rolling'] else 'ax-cpt'
            data_df['strategy'] = strategy
            data_df['n_block'] = data_df['n_block'].bfill()
            data_df['n_back'] = data_df['n_back'].bfill()
            data_df['responseTime'] = data_df['responseTime'].replace(1.5, pd.NA)
            data_df['click'] = data_df['responseTime'].notna().astype(int)
            
            
            #data_df.insert(0, 'sub', sub)
            #data_df.insert(1, 'strategy', strategy)
            all_data.append(data_df)
            found = True
            break
    if not found:
        print(f"No matching file for {sub} {strategy}")

# Concatenate all dataframes
dfa = pd.concat(all_data, ignore_index=True)
# change 9 to 'correct_rejection' 11 to 'miss' 12 to 'false_alarm' and 13 to 'hit'
dfa['feedback'] = dfa['feedback'].replace({
    9: 'correct_rejection',
    11: 'miss',
    12: 'false_alarm',
    13: 'hit'
})
dfa.to_csv('/Volumes/x9/INITIAL_DATABASE/dfa-trials-behave.csv', index=False)
display(dfa.head())

,task,n_block,n_back,trial,letter,responseTime,feedback,score,sub,strategy,click
0,ax-cpt,0.0,1.0,0.0,n,<NA>,correct_rejection,0,sub04,reactive,0
1,ax-cpt,0.0,1.0,1.0,c,<NA>,miss,0,sub04,reactive,0
2,ax-cpt,0.0,1.0,2.0,n,<NA>,correct_rejection,0,sub04,reactive,0
3,ax-cpt,0.0,1.0,3.0,c,<NA>,miss,0,sub04,reactive,0
4,ax-cpt,0.0,1.0,4.0,m,<NA>,correct_rejection,0,sub04,reactive,0


In [8]:
#combine task with strategy columns
dfa['task_strategy'] = dfa['task'] + '-' + dfa['strategy']
dfa['task_strategy'].unique()

array(['ax-cpt-reactive', 'n-back-static', 'ax-cpt-proactive',
       'n-back-rolling'], dtype=object)